In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Read data as dataframe
df = pd.read_csv("/home/donajialej/data/3-Simon/promoterDHS_distalDHS_pairs.csv")

In [ ]:
from Bio import SeqIO

# Define a function to convert a chromosome string to a genomic sequence
def chrom2seq(chrom):
    """Convert chromosome string "chrom" to a genomic sequence"""

    return list(SeqIO.parse(
        "/home/donajialej/data/3-Simon/hg38.analysisSet.chroms/%s.fa" % chrom, "fasta"))[0].seq.upper()

In [ ]:
# list of chromosomes
CHROMS = ["chr1", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", 
          "chr18", "chr19", "chr2", "chr20", "chr21", "chr22", "chr3", "chr4", "chr5", "chr6", 
          "chr7", "chr8", "chr9", "chrM", "chrX", "chrY"]

# Make dictionary
CHROM2SEQ = dict(zip(CHROMS,[chrom2seq(chrom) for chrom in CHROMS]))

In [ ]:
seq = CHROM2SEQ['chr1'][925589:925790]
print(seq)

In [ ]:
# Define a function to create a one-hot encoding for a genomic sequence
base2row = {'A':0,'C':1,'G':2,'T':3}
def seq2onehot(seq):
    """Create a one-hot encoding of a nucleotide sequence"""
    
    A = np.zeros((len(seq),4),dtype=bool)
    for i, base in enumerate(seq):
        if base=='N':
            continue
        A[i,base2row[base]] = True
    return A

In [ ]:
seq1h = seq2onehot(seq)
np.shape(seq1h)
print(seq1h[:5,:])
seq[:5]

In [ ]:
# Define a function to find the reverse compliment of a one-hot encoding
def onehot2rc(onehot):
    """Find the one-hot encoding of the reverse compliment
    of the genomic sequence with one-hot encoding onehot"""
    
    A = np.flipud(onehot) # First reverse
    A = np.fliplr(A) # Then take compliment (i.e. switch A&T and C&G)
    #B = np.zeros(np.shape(A),dtype=bool)
    #B[:,0], B[:,1], B[:,2], B[:,3] = A[:,1], A[:,0], A[:,3], A[:,2]
    return A

In [ ]:
A = seq1h[:5,:]
B = onehot2rc(A)
print(A)
print(B)

In [ ]:

# Sample the same number of linked==1 class as linked==0 class

idx1 = np.where(df["linked"])[0]
idx0 = np.where(df["linked"]==0)[0]
np.random.shuffle(idx1)
np.random.shuffle(idx0)

nsamples_per_class = 20000
print(np.shape(idx1))
print(np.shape(idx0))
idx1 = idx1[:nsamples_per_class]
idx0 = idx0[:nsamples_per_class]
print(np.shape(idx1))
print(np.shape(idx0))

# Save these indices
np.savez("/home/donajialej/BCdata_project3/SampleIdx.npz",
         idx1=idx1, idx0=idx0)
"""
# Load idx1 and idx0 if saved
sample_idx = np.load("/home/joshscurll/bcdata_Altius_Project/BCdata_project3/SampleIdx.npz")
idx1, idx0 = sample_idx['idx1'], sample_idx['idx0']
"""

In [ ]:
np.concatenate([idx1, idx0])

In [ ]:
# Take a sample
idx10 = np.concatenate([idx1,idx0])
np.random.shuffle(idx10)
newdf = df.iloc[idx10]
del df
df = newdf
del newdf

df = df.sample(n=20000) # Comment this out when loading indices!

In [ ]:
df.head()

In [ ]:
# Create new columns for promoter length and distal length
df["promoter_len"] = df["promoterDHSend"]-df["promoterDHSstart"]
df["distal_len"] = df["distalDHSend"]-df["distalDHSstart"]

# Plot histogram of sequence lengths
plt.figure
plt.hist(df["promoter_len"],bins=30,color="r",alpha=0.4,label="promoter")
plt.hist(df["distal_len"],bins=30,color="b",alpha=0.4,label="distal")
plt.legend()
plt.xlabel("base pair")
plt.ylabel("count")
plt.show()

In [ ]:
# Set a maximum sequence length and filter out longer sequences
seqlength = 2000
df = df[(df["promoter_len"]<seqlength) & (df["distal_len"]<seqlength)]

In [ ]:
# Initialize one-hot encoding arrays (X)
N = df.shape[0]
encoding_promoter = np.zeros((N,seqlength,2,4),dtype=bool)
encoding_distal = np.zeros((N,seqlength,2,4),dtype=bool)

# Initialize Y
Y = np.zeros((N,1))

In [ ]:
# Define X (one-hot encoding) and Y (linked) for Keras
row_no = 0
for i,row in df.iterrows():
    seq_promoter = CHROM2SEQ[row.chr][row.promoterDHSstart:row.promoterDHSend]
    seq_distal = CHROM2SEQ[row.chr][row.distalDHSstart:row.distalDHSend]
    try:
        onehot_promoter = seq2onehot(seq_promoter)
        onehot_distal = seq2onehot(seq_distal)
        encoding_promoter[row_no,:row.promoter_len,0,:] = onehot_promoter
        encoding_distal[row_no,:row.distal_len,0,:] = onehot_distal
        encoding_promoter[row_no,:row.promoter_len,1,:] = onehot2rc(onehot_promoter)
        encoding_distal[row_no,:row.distal_len,1,:] = onehot2rc(onehot_distal)
        Y[row_no] = row.linked
        row_no += 1
    except KeyError:
        print(seq_promoter)
        print(seq_distal)
        break
        
del onehot_promoter
del onehot_distal

In [ ]:
print(np.shape(encoding_promoter))
print(np.shape(encoding_distal))
print(np.shape(Y))

In [ ]:
from sklearn.model_selection import train_test_split

"""Test whether we can predict promoter and distal sequences using model"""

X = np.zeros((2*N,seqlength,2,4),dtype=bool)
Y = np.zeros((2*N,1),dtype=bool)
X[:N,:,:,:] = encoding_promoter
X[N:,:,:,:] = encoding_distal
Y[:N] = np.ones((N,1),dtype=bool)

# Create training, development and test data sets

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25)

X_train, X_dev, y_train, y_dev = train_test_split(
    X_train, y_train, test_size=0.33)

In [ ]:
import keras 
from keras import backend as K

import numpy as np

from keras.layers import Dense, Dropout, Input
from keras.layers import Conv1D, Conv2D, GlobalMaxPooling1D, GlobalMaxPooling2D, MaxPooling2D
from keras.models import Model
from keras.constraints import non_neg
from keras import regularizers as regs

from keras.utils import to_categorical

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Define some functions for the model

def reluConv2D(x, filters, kernel_size, name):
    return Conv2D(filters=filters, kernel_size=(kernel_size,1),
                 activation='relu', padding='same', name=name,
                  kernel_constraint=non_neg(),
                 kernel_regularizer=regs.l1(0.001))(x)

def gmp2d(x):
    return GlobalMaxPooling2D()(x)

"""
def columnwise_mp(x):
    return MaxPooling2D(pool_size=(int(np.shape(x)[1]),1))(x)
    #return MaxPooling2D(pool_size=(2000,1))(x)

def reluConv1D(x, filters, kernel_size, name):
    return Conv1D(filters=filters, kernel_size=kernel_size, 
                  activation='relu', padding='same', name=name)(x)

def gmp1d(x):
    return GlobalMaxPooling1D()(x)
"""

def convMP(x, filters, kernel_size, name):
    return gmp2d(reluConv2D(x, filters, kernel_size, name))

In [ ]:
np.shape(encoding_promoter)

In [ ]:
# Specify the model
onehot = Input(shape=(seqlength,2,4),name='promoterDHSoh')
fingerprint = convMP(onehot,24,6,'ConvPromoterDHS')
prob_association = Dense(1,activation='sigmoid',name='logistic_regr')(fingerprint)

In [ ]:
model = Model(inputs=onehot,outputs=prob_association)
model.summary()

In [ ]:
# Get the percentage of nonzeros in Y
pc = len(np.where(Y)[0]) / len(Y)
print(pc)

In [ ]:
from keras.optimizers import Adam
from keras import metrics
from keras.callbacks import ModelCheckpoint


# Compile the model
model.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=['mse','accuracy'])

In [ ]:
# ALEJANDRA: remove the weight given that it's 50% already
model.fit(X_train,y_train,batch_size=256,epochs=100,
          validation_data=[X_dev,y_dev],
          callbacks=[EarlyStopping(patience=3),
                     ModelCheckpoint('T_RC_12nf_lambda001_CP', monitor='val_loss', verbose=0, save_best_only=True)])

In [ ]:
model.save("T_RC_12nf_lambda001.h5")

In [ ]:
# accuracy and lost
# filters
# weblogs
# ROC
# testing scores

plt.rcParams.update({'font.size': 16})

In [ ]:
"""
Visualizing the filters
"""
filters=model.get_layer('ConvPromoterDHS').get_weights()[0]
fig, axs = plt.subplots(int(np.ceil(np.shape(filters)[3]/4)),4, figsize=(15, 12), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(np.shape(filters)[3]):
    im=axs[i].imshow(filters[:,0,:,i].T)
    plt.colorbar(im,ax=axs[i])
#plt.title('Filters')
plt.show()

In [ ]:
import sequenceLogoPlot

In [ ]:
temp=[[ 0.02247014831444764,
               0.057903843733384308,
               0.10370837683591219,
               0.24803586793255664],
             [ 0.046608227674354567,
               0.048827667087419063,
               0.084338697696451109,
               0.92994511407402669]]
np.shape(temp)

In [ ]:
sequenceLogoPlot.modified_draw_logo(temp, ['A','C','G','T'])

In [ ]:
np.shape(filters[:,0,:,0])

In [ ]:
print(filters[:,0,:,0])

In [ ]:
for index, scores in enumerate(filters[:,0,:,0]):
    for indey, score in enumerate(scores):
        print(indey)
        print(score)